In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import array
from pyspark.sql.functions import create_map, lit
from pyspark.sql.functions import struct
from pyspark.sql.types import StringType

In [ ]:

spark = SparkSession.builder.appName("Company").getOrCreate()

data = [
    ("James", "Sales", 3000, 28, 2),
    ("Michael", "Sales", 4600, 35, 5),
    ("Robert", "Sales", 4100, 29, 3),
    ("Maria", "Finance", 3000, 32, 4),
    ("Rina", "Finance", 4200, 41, 10),
    ("John", "Finance", 3900, 27, 2),
    ("Tono", "HR", 2800, 26, 1),
    ("Sinta", "HR", 3500, 33, 7),
    ("Budi", "HR", 4000, 38, 12),
    ("Agus", "IT", 5000, 30, 6),
    ("Ayu", "IT", 5500, 34, 8),
    ("Andi", "IT", 6000, 40, 10),
    ("Putri", "IT", 4500, 27, 3),
    ("Yoga", "Marketing", 3100, 29, 2),
    ("Dewi", "Marketing", 3700, 36, 6),
    ("Riko", "Marketing", 3300, 31, 4),
    ("Tari", "Marketing", 4200, 42, 15),
    ("Dian", "Sales", 3900, 26, 1),
    ("Eko", "Finance", 4700, 37, 9),
    ("Wulan", "HR", 3600, 28, 4)
]

columns = ["EmployeeName", "Department", "Salary", "Age", "YearsAtCompany"]

df = spark.createDataFrame(data, schema=columns)

df.show(20)


+------------+----------+------+---+--------------+
|EmployeeName|Department|Salary|Age|YearsAtCompany|
+------------+----------+------+---+--------------+
|       James|     Sales|  3000| 28|             2|
|     Michael|     Sales|  4600| 35|             5|
|      Robert|     Sales|  4100| 29|             3|
|       Maria|   Finance|  3000| 32|             4|
|        Rina|   Finance|  4200| 41|            10|
|        John|   Finance|  3900| 27|             2|
|        Tono|        HR|  2800| 26|             1|
|       Sinta|        HR|  3500| 33|             7|
|        Budi|        HR|  4000| 38|            12|
|        Agus|        IT|  5000| 30|             6|
|         Ayu|        IT|  5500| 34|             8|
|        Andi|        IT|  6000| 40|            10|
|       Putri|        IT|  4500| 27|             3|
|        Yoga| Marketing|  3100| 29|             2|
|        Dewi| Marketing|  3700| 36|             6|
|        Riko| Marketing|  3300| 31|             4|
|        Tar

In [ ]:
df.groupBy("Department").agg(
    F.count("*").alias("TotalEmployees"),
    F.avg("Salary").alias("AvgSalary"),
    F.max("Salary").alias("MaxSalary"),
    F.min("Salary").alias("MinSalary"),
    F.avg("Age").alias("AvgAge")
).show()

for dept in df.select("Department").distinct().collect():
    dept_name = dept["Department"]
    print(f"\n=== Data Karyawan Departemen: {dept_name} ===")
    df.filter(df["Department"] == dept_name).select(
        "Department",
        "EmployeeName",
        "Salary",
    ).show()


+----------+--------------+---------+---------+---------+------+
|Department|TotalEmployees|AvgSalary|MaxSalary|MinSalary|AvgAge|
+----------+--------------+---------+---------+---------+------+
|     Sales|             4|   3900.0|     4600|     3000|  29.5|
|        HR|             4|   3475.0|     4000|     2800| 31.25|
|   Finance|             4|   3950.0|     4700|     3000| 34.25|
|        IT|             4|   5250.0|     6000|     4500| 32.75|
| Marketing|             4|   3575.0|     4200|     3100|  34.5|
+----------+--------------+---------+---------+---------+------+


=== Data Karyawan Departemen: Sales ===
+----------+------------+------+
|Department|EmployeeName|Salary|
+----------+------------+------+
|     Sales|       James|  3000|
|     Sales|     Michael|  4600|
|     Sales|      Robert|  4100|
|     Sales|        Dian|  3900|
+----------+------------+------+


=== Data Karyawan Departemen: HR ===
+----------+------------+------+
|Department|EmployeeName|Salary|
+---

In [ ]:
df = df.withColumn("Bonus", df["Salary"] * 0.15)
df = df.withColumn("TotalSalary", df["Salary"] + df["Bonus"])
df.show(20)

df1 = df.withColumn("Experience", array("Department", "YearsAtCompany")).orderBy("Department", "YearsAtCompany")
df1.show(truncate=False)

df2 = df.withColumn("Profile", create_map(lit("EmployeeName"), df["EmployeeName"], lit("Department"), df["Department"])).orderBy("Department")
df2.show(truncate=False)

df3 = df.withColumn("ComponentSalary", struct("Salary", "Bonus")).orderBy("Department")
df3.show(truncate=False)

+------------+----------+------+---+--------------+-----+-----------+
|EmployeeName|Department|Salary|Age|YearsAtCompany|Bonus|TotalSalary|
+------------+----------+------+---+--------------+-----+-----------+
|       James|     Sales|  3000| 28|             2|450.0|     3450.0|
|     Michael|     Sales|  4600| 35|             5|690.0|     5290.0|
|      Robert|     Sales|  4100| 29|             3|615.0|     4715.0|
|       Maria|   Finance|  3000| 32|             4|450.0|     3450.0|
|        Rina|   Finance|  4200| 41|            10|630.0|     4830.0|
|        John|   Finance|  3900| 27|             2|585.0|     4485.0|
|        Tono|        HR|  2800| 26|             1|420.0|     3220.0|
|       Sinta|        HR|  3500| 33|             7|525.0|     4025.0|
|        Budi|        HR|  4000| 38|            12|600.0|     4600.0|
|        Agus|        IT|  5000| 30|             6|750.0|     5750.0|
|         Ayu|        IT|  5500| 34|             8|825.0|     6325.0|
|        Andi|      

In [ ]:
windowSpec = Window.partitionBy("Department").orderBy(F.desc("Salary"))

df_ranked = df.withColumn("DenseRank", F.dense_rank().over(windowSpec)) \
              .withColumn("RunningTotal", F.sum("Salary").over(windowSpec))
df_ranked.show(20)

def salary_category(salary):
    if salary < 3500:
        return "Low"
    elif salary < 5000:
        return "Medium"
    else:
        return "High"

salary_udf = F.udf(salary_category, StringType())
df_catSalary = df.withColumn("SalaryCategory", salary_udf(df["Salary"]))
df_catSalary.show(20)

df_catYears = df.withColumn("YearsCategory",
    F.when(df["YearsAtCompany"] < 5, "Baru")
     .when((df["YearsAtCompany"] >= 5) & (df["YearsAtCompany"] < 10), "Menengah")
     .otherwise("Senior")
)

df_catYears.select("EmployeeName", "Department", "Age", "YearsAtCompany", "YearsCategory").orderBy("Department").show(20)


+------------+----------+------+---+--------------+-----+-----------+---------+------------+
|EmployeeName|Department|Salary|Age|YearsAtCompany|Bonus|TotalSalary|DenseRank|RunningTotal|
+------------+----------+------+---+--------------+-----+-----------+---------+------------+
|         Eko|   Finance|  4700| 37|             9|705.0|     5405.0|        1|        4700|
|        Rina|   Finance|  4200| 41|            10|630.0|     4830.0|        2|        8900|
|        John|   Finance|  3900| 27|             2|585.0|     4485.0|        3|       12800|
|       Maria|   Finance|  3000| 32|             4|450.0|     3450.0|        4|       15800|
|        Budi|        HR|  4000| 38|            12|600.0|     4600.0|        1|        4000|
|       Wulan|        HR|  3600| 28|             4|540.0|     4140.0|        2|        7600|
|       Sinta|        HR|  3500| 33|             7|525.0|     4025.0|        3|       11100|
|        Tono|        HR|  2800| 26|             1|420.0|     3220.0| 

In [ ]:
from pyspark.sql import functions as SparkFuncs
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Anime").getOrCreate()

dataframe = spark.read.csv("/kaggle/dataset/archive.csv", header=True, inferSchema=True)
dataframe.show(dataframe.count())

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

top_score = dataframe.filter(dataframe["score"].isNotNull()) \
    .orderBy(F.desc("score"), F.desc("scored_by")) \
    .select("title_name", "score", "scored_by", "ranked") \
    .limit(10)

print("\n=== Top 10 Anime Skor Tertinggi ===")
top_score.show(truncate=False)


=== Top 10 Anime Skor Tertinggi ===
+----------------------------------+-----+---------+------+
|title_name                        |score|scored_by|ranked|
+----------------------------------+-----+---------+------+
|Sousou no Frieren                 |9.3  |696616.0 |1     |
|Fullmetal Alchemist: Brotherhood  |9.1  |2232236.0|2     |
|Steins;Gate                       |9.07 |1472818.0|3     |
|Shingeki no Kyojin Season 3 Part 2|9.05 |1710556.0|4     |
|Gintama°                          |9.05 |265163.0 |6     |
|Gintama: The Final                |9.05 |81382.0  |5     |
|One Piece Fan Letter              |9.04 |86983.0  |7     |
|Hunter x Hunter (2011)            |9.03 |1893615.0|8     |
|Gintama'                          |9.02 |250201.0 |10    |
|Gintama': Enchousen               |9.02 |174445.0 |11    |
+----------------------------------+-----+---------+------+



In [ ]:

type_distribution = dataframe.groupBy("item_type").count().orderBy(F.desc("count"))
print("\n=== Distribusi Tipe Anime ===")
type_distribution.show(truncate=False)


=== Distribusi Tipe Anime ===
+----------+-----+
|item_type |count|
+----------+-----+
|TV        |8322 |
|Movie     |4868 |
|OVA       |4177 |
|ONA       |4027 |
|Music     |3919 |
|Special   |1762 |
|TV Special|751  |
|CM        |468  |
|PV        |260  |
|Unknown   |72   |
|3         |1    |
|5         |1    |
|537       |1    |
|241       |1    |
|152       |1    |
|8         |1    |
|16        |1    |
|0         |1    |
|149       |1    |
+----------+-----+



In [ ]:
airing_date_clean = F.when(F.col("airing_date").contains(" to "),
                             F.regexp_extract(F.col("airing_date"), "(.+?) to", 1)).otherwise(F.col("airing_date"))

dataframe_with_year = dataframe.withColumn("year", F.year(F.to_timestamp(airing_date_clean, "MMM d, yyyy")))


max_year = dataframe_with_year.agg(F.max("year")).collect()[0][0]
min_year = max_year - 5


source_per_year = dataframe_with_year.groupBy("year", "source") \
    .agg(F.count("*").alias("total_anime")) \
    .orderBy("year", F.desc("total_anime"))

print("\n=== Jumlah Anime Berdasarkan Source per Tahun ===")
source_per_year.filter((F.col("year") >= min_year) & (F.col("year") <= max_year)).show(50,truncate=False)


=== Jumlah Anime Berdasarkan Source per Tahun ===
+----+----------------------------------+-----------+
|year|source                            |total_anime|
+----+----------------------------------+-----------+
|2020|Original                          |134        |
|2020|Manga                             |44         |
|2020|Game                              |26         |
|2020|Other                             |24         |
|2020|[]                                |23         |
|2020|Novel                             |22         |
|2020|Web manga                         |20         |
|2020|Unknown                           |17         |
|2020|Visual novel                      |14         |
|2020|Web novel                         |7          |
|2020|Light novel                       |7          |
|2020|Picture book                      |6          |
|2020|Mixed media                       |6          |
|2020| ""Kadokawa Media House""         |6          |
|2020|"[""Funimation""]"       

In [ ]:
df_genre = dataframe_with_year.withColumn("genre_array", F.split(F.col("genres"), ", "))
df_genre = df_genre.withColumn("genre", F.explode("genre_array"))

genre_per_year = df_genre.groupBy("year", "genre") \
    .agg(F.count("*").alias("total_anime")) \
    .orderBy("year", F.desc("total_anime"))

print(f"\n=== Tren Genre Anime per Tahun ({min_year}–{max_year}) ===")
genre_per_year.filter((F.col("year") >= min_year) & (F.col("year") <= max_year)).show(50, truncate=False)


=== Tren Genre Anime per Tahun (2020–2025) ===
+----+-------------------------------------------+-----------+
|year|genre                                      |total_anime|
+----+-------------------------------------------+-----------+
|2020|"[""Comedy""]"                             |56         |
|2020|"[""Action""                               |56         |
|2020|"[""Comedy""                               |48         |
|2020|[]                                         |39         |
|2020|"[""Adventure""                            |23         |
|2020|"[""Slice of Life""]"                      |22         |
|2020|"[""Hentai""]"                             |22         |
|2020|Manga                                      |19         |
|2020|"[""Fantasy""]"                            |17         |
|2020|"[""Fantasy""                              |12         |
|2020|Original                                   |12         |
|2020|"[""Action""]"                             |10         |
|2020|"

In [ ]:
genre_count = df_genre.groupBy("genre") \
    .agg(F.count("*").alias("total_anime")) \
    .orderBy(F.desc("total_anime"))

print("\n=== Jumlah Anime per Genre ===")
genre_count.show(50, truncate=False)


=== Jumlah Anime per Genre ===
+---------------------------------+-----------+
|genre                            |total_anime|
+---------------------------------+-----------+
|[]                               |5880       |
|"[""Action""                     |2625       |
|"[""Comedy""]"                   |2216       |
|"[""Comedy""                     |1840       |
|"[""Adventure""                  |1427       |
|"[""Fantasy""]"                  |1362       |
|"[""Hentai""]"                   |959        |
|Manga                            |943        |
|Original                         |779        |
|"[""Avant Garde""]"              |768        |
|"[""Drama""]"                    |644        |
|"[""Slice of Life""]"            |636        |
|"[""Drama""                      |497        |
|"[""Action""]"                   |486        |
|"[""Adventure""]"                |413        |
|"[""Sci-Fi""]"                   |341        |
|"[""Sports""]"                   |284        |
|"[""Fan